# Database Population

In this notebook, we connect to a relational database and insert cleaned and structured data. This process enables organized storage and facilitates efficient access for future analysis or reporting tasks.

In [ ]:
import pandas as pd
import mysql.connector as sql
import csv
import os

## Dynamic CSV Loading from /data/tables

In [ ]:
# Paths to CSV files
paths = {
    # 'main_dataset': 'data/processed/main_dataset.csv',
    'department': 'data/tables/department.csv',
}

# Dictionary to store loaded DataFrames
dfs = {}

for name, path in paths.items():
    try:
        # Try loading with UTF-8 encoding
        df = pd.read_csv(path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # Fallback to Latin1 encoding if UTF-8 fails
            df = pd.read_csv(path, encoding='latin1')
        except Exception as e:
            print(f"❌ Error loading '{name}' from '{path}': {e}")
            df = None
    dfs[name] = df

# Print summary of loaded files
print("\n📦 Summary of loaded files:")
for name, df in dfs.items():
    if df is not None:
        print(f"✅ '{name}' loaded successfully. Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    else:
        print(f"⚠️ '{name}' failed to load.")


## Accessing DataFrames Loaded in the Previous Notebook

The DataFrames are stored in the dictionary dfs. You can access each DataFrame to run any pandas commands as usual.

In [ ]:
dfs['department'][['COD_ESTADO_DEPTO', 'ESTADO_DEPTO']].head()

## Establishing a MySQL Connection

To connect to a MySQL server, you need the access credentials (host, user, password, and database). We use the mysql.connector.connect() method to establish the connection.

In [ ]:
import mysql.connector as sql

# Connect to the database
connection = sql.connect(
    host="localhost",      # Change to your database host
    port=3306,             # Change to your database port
    user="root",           # Change to your database user
    password="root",   # Change to your user's password
    database="database"  # Change to your database name
)

if connection.is_connected():
    print("Successfully connected to the database")

# connection.close()


## Populating the conflicto_armado Database

In the following cells, data is loaded into the database from their respective DataFrames.

### Tabla: Department

In [ ]:
cursor = connection.cursor()

df_department = dfs['department']

# Iterate over the DataFrame rows
for _, row in df_department.iterrows():
    try:
        query = """
            INSERT INTO Department (id_department, name_department)
            VALUES (%s, %s)
        """
        values = (int(row['COD_ESTADO_DEPTO']), row['ESTADO_DEPTO'])  # Ensure integer type
        cursor.execute(query, values)
    except Exception as e:
        print(f"❌ Error inserting record with ID {row['COD_ESTADO_DEPTO']}: {e}")

# Commit the changes
connection.commit()
print("✅ Insertion into Department completed.")

cursor.close()
